In [ ]:
# -*- coding: utf-8 -*-
"""MSML612_TrafficYOLO_weight.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1CQc2qjUYp-nas_pBBK9wWwqDBKWlg2Xe
"""

In [1]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15298, done.
remote: Total 15298 (delta 0), reused 0 (delta 0), pack-reused 15298
Receiving objects: 100% (15298/15298), 13.68 MiB | 14.05 MiB/s, done.
Resolving deltas: 100% (10398/10398), done.


# Data prep


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data Download
!wget https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip

--2021-08-10 04:14:47--  https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip
Resolving sid.erda.dk (sid.erda.dk)... 130.225.104.13
Connecting to sid.erda.dk (sid.erda.dk)|130.225.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1661866983 (1.5G) [application/zip]
Saving to: ‘FullIJCNN2013.zip.1’

FullIJCNN2013.zip.1 100%[===================>]   1.55G  10.9MB/s    in 2m 31s  

2021-08-10 04:17:19 (10.5 MB/s) - ‘FullIJCNN2013.zip.1’ saved [1661866983/1661866983]



In [ ]:
!unzip FullIJCNN2013.zip

Archive:  FullIJCNN2013.zip
   creating: FullIJCNN2013/
   creating: FullIJCNN2013/00/
  inflating: FullIJCNN2013/00000.ppm  
  inflating: FullIJCNN2013/00001.ppm  
  inflating: FullIJCNN2013/00002.ppm  
  inflating: FullIJCNN2013/00003.ppm  
  inflating: FullIJCNN2013/00004.ppm  
  inflating: FullIJCNN2013/00005.ppm  
  inflating: FullIJCNN2013/00006.ppm  
  inflating: FullIJCNN2013/00007.ppm  
  inflating: FullIJCNN2013/00008.ppm  
  inflating: FullIJCNN2013/00009.ppm  
  inflating: FullIJCNN2013/00010.ppm  
  inflating: FullIJCNN2013/00011.ppm  
  inflating: FullIJCNN2013/00012.ppm  
  inflating: FullIJCNN2013/00013.ppm  
  inflating: FullIJCNN2013/00014.ppm  
  inflating: FullIJCNN2013/00015.ppm  
  inflating: FullIJCNN2013/00016.ppm  
  inflating: FullIJCNN2013/00017.ppm  
  inflating: FullIJCNN2013/00018.ppm  
  inflating: FullIJCNN2013/00019.ppm  
  inflating: FullIJCNN2013/00020.ppm  
  inflating: FullIJCNN2013/00021.ppm  
  inflating: FullIJCNN2013/00022.ppm  
  inflating: Ful

In [ ]:
!ls -al ./FullIJCNN2013/gt.txt 

-rw-r--r-- 1 root root 35282 Jul  9  2013 ./FullIJCNN2013/gt.txt


In [ ]:
import pandas as pd
import os
import cv2

In [ ]:
# train and test dataset
test_percentage = 0.2
file_name = './FullIJCNN2013/gt.txt'

In [ ]:
annotation = pd.read_csv(file_name,
                         names=['ImageId', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId'],
                         sep=';')

In [ ]:
annotation.head()

,ImageId,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId
0,00000.ppm,774,411,815,446,11
1,00001.ppm,983,388,1024,432,40
2,00001.ppm,386,494,442,552,38
3,00001.ppm,973,335,1031,390,13
4,00002.ppm,892,476,1006,592,39


In [ ]:
Prohibitory = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]
Danger = [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Mandatory = [33, 34, 35, 36, 37, 38, 39, 40]
Other = [6, 12, 13, 14, 17, 32, 41, 42]

categories = ['Prohibitory', 'Danger', 'Mandatory', 'Other']

In [ ]:
annotation['CategoryId'] = 0

In [ ]:
# create class id for each category  
annotation.loc[annotation['ClassId'].isin(Prohibitory), 'CategoryId'] = 0
annotation.loc[annotation['ClassId'].isin(Danger), 'CategoryId'] = 1
annotation.loc[annotation['ClassId'].isin(Mandatory), 'CategoryId'] = 2
annotation.loc[annotation['ClassId'].isin(Other), 'CategoryId'] = 3

In [ ]:
# center X and Y
annotation['centerX'] = (annotation['Roi.X2']+annotation['Roi.X1']) / 2 
annotation['centerY'] = (annotation['Roi.Y2']+annotation['Roi.Y1']) / 2 
annotation['width'] = (annotation['Roi.X2'] - annotation['Roi.X1']) 
annotation['height'] = (annotation['Roi.Y2'] - annotation['Roi.Y1']) 

In [ ]:
annotation.head()

,ImageId,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,CategoryId,centerX,centerY,width,height
0,00000.ppm,774,411,815,446,11,1,794.5,428.5,41,35
1,00001.ppm,983,388,1024,432,40,2,1003.5,410.0,41,44
2,00001.ppm,386,494,442,552,38,2,414.0,523.0,56,58
3,00001.ppm,973,335,1031,390,13,3,1002.0,362.5,58,55
4,00002.ppm,892,476,1006,592,39,2,949.0,534.0,114,116


In [ ]:
image_path = './FullIJCNN2013'

In [ ]:
os.chdir(image_path)

In [ ]:
paths = []
for current_dir, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.ppm'):
            image_ppm = cv2.imread(file)
            h, w = image_ppm.shape[:2]
            image_name = file.split('.')[0]
            txt_file = image_name + '.txt'
            image_jpg = image_name + '.jpg'
            cv2.imwrite(image_jpg, image_ppm)
            txt = open(txt_file, 'w')
            objects = annotation.loc[annotation['ImageId'] == file].copy()
            for index, data in objects.iterrows():
                line = str(data['CategoryId']) + ' ' + str(data['centerX']/w) + ' ' + str(data['centerY']/h) + ' ' + str(data['width']/w) + ' ' + str(data['height']/h) + '\n'
                txt.write(line) 
            image_path = image_jpg + '\n'
            paths.append(image_path)
            print(image_jpg, ' is crated')             

00204.jpg  is crated
00351.jpg  is crated
00007.jpg  is crated
00466.jpg  is crated
00059.jpg  is crated
00109.jpg  is crated
00137.jpg  is crated
00136.jpg  is crated
00218.jpg  is crated
00777.jpg  is crated
00893.jpg  is crated
00162.jpg  is crated
00780.jpg  is crated
00363.jpg  is crated
00428.jpg  is crated
00198.jpg  is crated
00539.jpg  is crated
00242.jpg  is crated
00040.jpg  is crated
00698.jpg  is crated
00072.jpg  is crated
00336.jpg  is crated
00443.jpg  is crated
00323.jpg  is crated
00724.jpg  is crated
00870.jpg  is crated
00299.jpg  is crated
00096.jpg  is crated
00279.jpg  is crated
00417.jpg  is crated
00676.jpg  is crated
00589.jpg  is crated
00289.jpg  is crated
00775.jpg  is crated
00166.jpg  is crated
00623.jpg  is crated
00157.jpg  is crated
00140.jpg  is crated
00277.jpg  is crated
00283.jpg  is crated
00891.jpg  is crated
00615.jpg  is crated
00006.jpg  is crated
00899.jpg  is crated
00459.jpg  is crated
00009.jpg  is crated
00065.jpg  is crated
00436.jpg  is

In [ ]:
# split train and test dataset
num_test = int(len(paths) * test_percentage)
paths_test = paths[:num_test]
paths_train = paths[num_test:]

# create train.txt
with open('train.txt', 'w') as train_txt:
    for path in paths_train:
        train_txt.write(path)

# create test.txt
with open('test.txt', 'w') as test_txt:
    for path in paths_test:
        test_txt.write(path)

In [ ]:
# create a classes.names file
with open('classes.names', 'w') as names:
    for category in categories:
        names.write(category)
  
# create a traffic.data file
with open('traffic.data', 'w') as data:
    data.write('classes = ' + '1\n')
    data.write('train = ' + 'train.txt' + '\n')
    data.write('valid = ' +  'test.txt' + '\n')
    data.write('names = ' +  'classes.names' + '\n')
    data.write('backup = backup')

In [ ]:
# creat a backup folder in case the notebook crashed and save the last weight file
!mkdir backup

In [ ]:
#copy customized config file 
!cp /content/drive/MyDrive/dataset/traffic-sign-yolo.cfg ./

In [ ]:
#Clone Darknet repo
!git clone https://github.com/AlexeyAB/darknet 

fatal: destination path 'darknet' already exists and is not an empty directory.


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet/darknet


In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
!./darknet detector train traffic.data /content/data/yolov4-custom.cfg yolov4.conv.137 -dont_show -map

/bin/bash: /content/darknet: Is a directory


In [ ]:
# kick off training from where it last saved
!./darknet detector train traffic.data /content/data/yolov4-custom.cfg /content/drive/MyDrive/dataset/backup/yolov4-custom_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.897938), count: 6, class_loss = 0.000814, iou_loss = 10.139292, total_loss = 10.140106 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 1069395, rewritten_bbox = 0.088929 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.850072), count: 19, class_loss = 0.448771, iou_loss = 109.002037, total_loss = 109.450806 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.889167), count: 8, class_loss = 0.228785, iou_loss = 13.806115, total_loss = 14.034901 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 1069422, rewritten_bbox = 0.0

In [ ]:
# kick off training from where it last saved
!./darknet detector train traffic.data /content/data/yolov4-custom.cfg /content/drive/MyDrive/dataset/backup/yolov4-custom_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.820457), count: 2, class_loss = 0.000385, iou_loss = 0.475523, total_loss = 0.475908 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 20187, rewritten_bbox = 0.108981 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.861491), count: 10, class_loss = 0.265496, iou_loss = 42.457115, total_loss = 42.722610 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 20197, rewritten_bbox = 0.108927 % 
v